In [24]:
#import libraries
import pandas as pd
import numpy as np
import re

from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prade\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('data.csv')

C:\Users\prade\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df = df[['text','target']]

In [9]:
df.head()

,text,target
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44919 entries, 0 to 44918
Data columns (total 2 columns):
text      44919 non-null object
target    44919 non-null int64
dtypes: int64(1), object(1)
memory usage: 701.9+ KB


In [12]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [21]:
porter = PorterStemmer()
stop = stopwords.words('english')
def tokenizer_stemmer(text):
    temp = [porter.stem(word) for word in text.split()]
    return [w for w in temp if w not in stop]

test ='this is a random text with running people in a run with some html tags <h1> hello world</h1>'
result = tokenizer_stemmer(preprocessor(test))
print(result)

['thi', 'random', 'text', 'run', 'peopl', 'run', 'html', 'tag', 'hello', 'world']


In [23]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=preprocessor,
                       tokenizer= tokenizer_stemmer,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

y = df.target.values
X = tfidf.fit_transform(df.text)

In [25]:
X_train,X_test,y_train,y_test = train_test_split(X,y, shuffle=True, random_state=2, test_size=0.4)

In [28]:
clf = LogisticRegressionCV(cv=5,
                            scoring='accuracy',
                          random_state=1,
                          verbose=1,
                          n_jobs=-1,
                          max_iter=300)
clf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=300, multi_class='warn', n_jobs=-1, penalty='l2',
                     random_state=1, refit=True, scoring='accuracy',
                     solver='lbfgs', tol=0.0001, verbose=1)

In [31]:
#save the trained classifier
saved_model = open('saved_model_fake_news.sav','wb')
pickle.dump(clf,saved_model)
saved_model.close()

In [33]:
#load saved model
file = 'saved_model_fake_news.sav'
saved_clf = pickle.load(open(file,'rb'))

In [34]:
saved_clf.score(X_test,y_test)

C:\Users\prade\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.9940449688334817